Sklearn Pipeline test

In [4]:
import sklearn
import numpy as np
import pandas as pd
import os.path
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''),os.path.pardir)))


In [19]:
from datk.preprocessing import make_column_transformer, get_preprocessed_df




In [6]:

df = pd.read_csv("./train.csv")

df1 = df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
df11 = df.drop(['Name','Ticket','Cabin'],axis=1)

ID = ['PassengerId']

df1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
df1.isnull().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df1 = pd.get_dummies(df,dummy_na=True)

In [8]:
df1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
from sklearn.pipeline import make_pipeline,Pipeline,FeatureUnion
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer,make_column_selector


num_imputer = Pipeline([
        ("imputer", SimpleImputer(strategy="median",add_indicator=False)),
        
    ])
cat_ohe = Pipeline([
    ("cat_imputer", SimpleImputer(strategy='constant',fill_value='NA')),
    ('ohe',OneHotEncoder(dtype=np.int,handle_unknown='ignore'))
    ])
    

In [10]:
col_transformer = ColumnTransformer(
    [('imp', num_imputer, make_column_selector(dtype_include=np.number)),
     ('ohe', cat_ohe, make_column_selector(dtype_include=['object','category']))
    ], remainder='passthrough'
    )


In [11]:
col_transformer.fit(df1)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imp',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f92aaa05410>),
                                ('ohe',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(dtype=<class 'int'>,
                                                                handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f92aaa055d0>)])

In [12]:
col_label = col_transformer.transformers_[0][2] + list(col_transformer.transformers_[1][1].steps[1][1].get_feature_names(
    col_transformer.transformers_[1][2]))
col_label

['Survived',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_NA',
 'Embarked_Q',
 'Embarked_S']

In [13]:
df2 = pd.DataFrame(col_transformer.transform(df1),
    columns=col_label, # get column name
    index = df1.index
)

In [14]:
df2.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [16]:
df3 = get_preprocessed_df(df1,col_transformer,fit_flag=False)

df3.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [17]:
col_trans = make_column_transformer()

In [18]:
df4 = get_preprocessed_df(df1,col_trans,fit_flag=True)
df4.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0


In [25]:
col_trans.transformers_[1][1].steps[1][1].get_feature_names()

array(['x0_female', 'x0_male', 'x1_C', 'x1_NA', 'x1_Q', 'x1_S'],
      dtype=object)

In [205]:
import joblib
joblib.dump(col_transformer,'./col_transformer.bin')



['./col_transformer.bin']

In [206]:
ct = joblib.load('./col_transformer.bin')

In [208]:
df3 = pd.DataFrame(ct.transform(df1),
    columns=ct.transformers_[0][2], # get column name
    index = df1.index
)
df3.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [211]:
!rm col_transformer.bin

rm: col_transformer.bin: No such file or directory
Binary_Classifier_Notebook.ipynb  random-forest_classification.yaml
evaluate.py                       random-forest_regression.yaml
example_classification.ipynb      sklearn_pipeline.ipynb
example_regression.ipynb          test.csv
fit                               test_titanic.csv
fit.py                            train.csv
housing.csv                       train_titanic.csv
model_classification.yaml         xgb_classification
model_regression.yaml             xgb_model_classification.yaml
model_results                     xgb_model_regression.yaml
predict.py                        xgb_regression
